In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.iolib.summary2 import summary_col
import itertools

In [2]:
# Read only total_tract_population column from acs_data.csv
tract_pop = pd.read_csv(
    "../data/acs_data.csv",
    usecols=["tract", "total_tract_population"],
    dtype={"total_tract_population": np.float32, "tract": np.int32},
)
tract_pop.columns

Index(['tract', 'total_tract_population'], dtype='object')

# Run Regression 

In [3]:
# Load the data
dtype_dict = {
    "B01003_001E_adj_supply_any_avg_tract": np.float32,
    "B01003_001E_adj_supply_hr_avg_tract": np.float32,
    "B01003_001E_totcost_any_avg_tract": np.float32,
    "state": np.int32,
    "county": np.int32,
    "tract": np.int32,
    "median_household_income": np.float32,
    "gini_index": np.float32,
    "educational_attainment": np.float32,
    "employment_status": np.float32,
    "housing_tenure": np.float32,
    "poverty_status": np.float32,
    "commute_time": np.float32,
    "without_health_insurance": np.float32,
    "white": np.float32,
    "black": np.float32,
    "am_indian_alaska_native": np.float32,
    "asian": np.float32,
    "other_race": np.float32,
    "hispanic_latino": np.float32,
    "density": np.float32,
    "white_share_state": np.float32,
    "black_share_state": np.float32,
    "am_indian_alaska_native_share_state": np.float32,
    "asian_share_state": np.float32,
    "other_race_share_state": np.float32,
    "hispanic_latino_share_state": np.float32,
}


data = pd.read_csv(
    "../data/acs_data_all.csv", dtype=dtype_dict
)  # Replace with the path to your data file

# Merge tract_pop to data
data = data.merge(tract_pop, how="left", on="tract", validate="1:1")

# Scale median_household_income by 10,000
data["median_household_income"] = data["median_household_income"] / 10000

# Scale density by 10,000
data["density"] = data["density"] / 10000

# Rename the variables
data.rename(
    columns={
        "B01003_001E_adj_supply_any_avg_tract": "Average Supply of Any Provider",
        "B01003_001E_adj_supply_hr_avg_tract": "Average Supply of HR Provider",
        "B01003_001E_totcost_any_avg_tract": "Average Total Cost of Any Provider",
        "median_household_income": "Median Household Income",
        "gini_index": "Gini Index",
        "educational_attainment": "Educational Attainment",
        "employment_status": "Employment Status",
        "housing_tenure": "Housing Tenure",
        "poverty_status": "Poverty Status",
        "commute_time": "Commute Time",
        "without_health_insurance": "Without Health Insurance",
        "white": "Share of White Population Tract",
        "black": "Share of Black Population Tract",
        "am_indian_alaska_native": "Share of American Indian/Alaska Native Population Tract",
        "asian": "Share of Asian Population Tract",
        "other_race": "Share of Other Race Population Tract",
        "hispanic_latino": "Share of Hispanic/Latino Population Tract",
        "density": "Population Density",
        "tract_pop": "total_tract_population",
        "white_share_state": "Share of White Population",
        "black_share_state": "Share of Black Population",
        "am_indian_alaska_native_share_state": "Share of American Indian/Alaska Native Population",
        "asian_share_state": "Share of Asian Population",
        "other_race_share_state": "Share of Other Race Population",
        "hispanic_latino_share_state": "Share of Hispanic/Latino Population",
    },
    inplace=True,
)

# calculate state population
state_population = data.groupby("state")["total_tract_population"].transform("sum")

# create tract_weight
data["tract_weight"] = data["total_tract_population"] / state_population

In [4]:
data.head()

,Average Supply of Any Provider,Average Supply of HR Provider,Average Total Cost of Any Provider,state,county,tract,Median Household Income,Gini Index,Educational Attainment,Employment Status,...,Share of Hispanic/Latino Population Tract,Population Density,Share of White Population,Share of Black Population,Share of American Indian/Alaska Native Population,Share of Asian Population,Share of Other Race Population,Share of Hispanic/Latino Population,total_tract_population,tract_weight
0,0.024394,0.007030,140.768204,27,27001,1231966324,4.6548,0.4291,0.142012,0.455980,...,0.000150,0.000218,0.000379,2.696204e-06,0.000008,0.000031,0.000579,0.000008,2240.0,0.000404
1,0.277115,0.000000,49.739838,27,27001,1231966424,4.8580,0.3815,0.084266,0.442195,...,0.000067,0.000226,0.000369,1.258228e-06,0.000017,0.000002,0.000920,0.000004,2284.0,0.000411
2,0.481750,0.081783,147.445908,27,27001,1231966524,4.4543,0.4515,0.155082,0.498758,...,0.000240,0.004462,0.000596,5.032914e-06,0.000007,0.000023,0.000051,0.000013,3483.0,0.000627
3,0.420769,0.028768,91.856232,27,27001,1231966624,4.8708,0.4339,0.121503,0.485145,...,0.000134,0.000295,0.000491,4.493673e-06,0.000026,0.000000,0.000047,0.000007,2970.0,0.000535
4,0.009878,0.000000,59.393177,27,27001,1231986725,5.0110,0.4564,0.087353,0.438879,...,0.000033,0.000295,0.000337,7.189876e-07,0.000000,0.000025,0.000003,0.000002,1905.0,0.000343


In [5]:
def run_regression_analysis(covariates_list, outcome_variable, data, model_names, weights_variable="total_tract_population", scalar=1):
    regression_results = {}
    rmse_results = {}

    for covariates, model_name in zip(covariates_list, model_names):
        X = sm.add_constant(data[covariates])
        y = data[outcome_variable] * scalar

        # Check if weights_variable is provided
        if weights_variable:
            # Create weights array
            weights = data[weights_variable]
            model = sm.WLS(y, X, weights=weights)
        else:
            model = sm.OLS(y, X)

        result = model.fit()

        rmse = np.sqrt(mean_squared_error(y, result.predict()))

        # Use model name as a key to store results
        regression_results[model_name] = result
        rmse_results[model_name] = rmse

    return regression_results, rmse_results

### Access to Any Provider

In [6]:
# Define the covariates

# First set of covariates will include racial group shares
first_set = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
]

# Second set of covariates will include the first set plus educational attainment, median income
second_set = first_set + [
    "Median Household Income",
    "Educational Attainment",
]

# Third set of covariates will include the second set plus age, population density, and unemployment rate
third_set = second_set + [
    "Gini Index",
    "Employment Status",
    "Housing Tenure",
    "Poverty Status",
    "Commute Time",
    "Without Health Insurance",
]

# The fourth set of covariates will include the third set plus the population density
fourth_set = third_set + ["Population Density"]

# Combine all covariates into a list of lists
covariates_list = [first_set, second_set, third_set, fourth_set]

# Define the outcome variable
outcome_variable = "Average Supply of Any Provider"

# Define the scalar
scalar = 1  # Replace with your desired scalar

# Define model names
model_names = ["First Set", "Second Set", "Third Set", "Fourth Set"]

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list, outcome_variable, data, model_names, weights_variable="total_tract_population", scalar = scalar
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}"
    }
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [7]:
# Variables are in order
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Employment Status",
    "Housing Tenure",
    "Poverty Status",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    'const',
]

# Separate list for non-standard error metrics
metrics_list = ['N', 'R2', 'Adjusted R2', 'RMSE']

# Last non-empty row name
last_non_empty = ''

# New index
new_index = []

for name in summary_df1.index:
    if name == '':
        new_index.append(last_non_empty + '_se')
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [item for sublist in [[var, var + "_se"] for var in covariate_list_ordered] for item in sublist if item in summary_df1.index]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith('_se')]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: '' for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,-40.614,-221.123**,-362.278***,-351.354***
,(91.080),(90.738),(99.163),(101.234)
Share of American Indian/Alaska Native Population,-71.144,-91.310,-31.111,-36.246
,(200.966),(196.764),(213.012),(213.282)
Share of Asian Population,-3.604*,-4.098**,-3.589*,-3.542*
,(2.051),(1.993),(1.998),(2.000)
Share of Other Race Population,0.046,-0.167,-0.134,-0.114
,(0.414),(0.403),(0.399),(0.401)
Share of Hispanic/Latino Population,-116.290,-26.966,-300.128,-300.732
,(215.766),(210.555),(219.915),(219.977)


### High Rated Providers

In [8]:
# Define the outcome variable
outcome_variable = "Average Supply of HR Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list, outcome_variable, data, model_names, weights_variable="total_tract_population", scalar = scalar
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}"
    }
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [9]:
# Variables are in order of the regression table
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Employment Status",
    "Housing Tenure",
    "Poverty Status",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    'const',
]

# Separate list for non-standard error metrics
metrics_list = ['N', 'R2', 'Adjusted R2', 'RMSE']

# Last non-empty row name
last_non_empty = ''

# New index
new_index = []

for name in summary_df1.index:
    if name == '':
        new_index.append(last_non_empty + '_se')
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [item for sublist in [[var, var + "_se"] for var in covariate_list_ordered] for item in sublist if item in summary_df1.index]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith('_se')]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: '' for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,303.013***,241.060***,155.542***,88.352**
,(36.817),(33.475),(35.948),(35.512)
Share of American Indian/Alaska Native Population,-83.592,39.045,-32.637,-1.052
,(81.237),(72.592),(77.219),(74.817)
Share of Asian Population,0.025,-0.900,-0.996,-1.285*
,(0.829),(0.735),(0.724),(0.702)
Share of Other Race Population,0.269,0.104,0.138,0.014
,(0.168),(0.149),(0.145),(0.141)
Share of Hispanic/Latino Population,13.293,169.814**,47.578,51.289
,(87.220),(77.679),(79.722),(77.165)


### Total Cost

In [10]:
# Define the outcome variable
outcome_variable = "Average Total Cost of Any Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list, outcome_variable, data, model_names, weights_variable="total_tract_population", scalar = scalar
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}"
    }
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [11]:
# Variables are in order of the regression table
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Employment Status",
    "Housing Tenure",
    "Poverty Status",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    'const',
]

# Separate list for non-standard error metrics
metrics_list = ['N', 'R2', 'Adjusted R2', 'RMSE']

# Last non-empty row name
last_non_empty = ''

# New index
new_index = []

for name in summary_df1.index:
    if name == '':
        new_index.append(last_non_empty + '_se')
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [item for sublist in [[var, var + "_se"] for var in covariate_list_ordered] for item in sublist if item in summary_df1.index]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith('_se')]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: '' for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,175260.001***,197694.783***,181793.596***,153017.980***
,(20941.561),(15372.781),(16439.192),(16307.721)
Share of American Indian/Alaska Native Population,-243519.504***,-84020.898**,-62467.024*,-48939.990
,(46207.408),(33335.870),(35312.928),(34357.327)
Share of Asian Population,377.575,-476.930,-227.583,-351.717
,(471.539),(337.737),(331.173),(322.198)
Share of Other Race Population,252.505***,167.208**,190.945***,137.754**
,(95.281),(68.197),(66.175),(64.602)
Share of Hispanic/Latino Population,-42408.061,99004.885***,46559.625,48149.187
,(49610.300),(35672.327),(36457.289),(35435.860)
